In [56]:
import json
import os
import nltk
import random
import math
import re
from collections import OrderedDict

In [57]:
base = "C:\\Users\\RE-PC\\Desktop\\selected\\\Dataset\\"

'''
    dict_1 = {"Name of file" : dictonary}
    dictonary = {'data' : data_2 }
    dic_2 = {text : "string"}
    

'''
full_q = []
for x in os.listdir(base):
    with open(base+x ,'r') as f:
        d = json.load(f)
        x= x.replace(".json" , "")
        d=d[x]

        for i in d:
            each_list = i['data']
            q =""
            for i in each_list:
                q = q + i['text']
            full_q.append(q)

print(len(full_q))
print(full_q[0])

13784
Add another song to the Cita Romantica playlist. 


In [58]:
def process(full_q):
    ls = []
    for i in full_q:
        i = re.sub(r'\d' , '' , i)
        i = i.lower()
        i = nltk.word_tokenize(i)
        ls.append(i)
    full_q = ls
    return full_q

full_q =process(full_q)
print(full_q[0])

['add', 'another', 'song', 'to', 'the', 'cita', 'romantica', 'playlist', '.']


In [59]:

test_full_q = []
for i in range(2000):
    x = random.choice(full_q)
    test_full_q.append(x);
    full_q.remove(x)
print(len(test_full_q))
print(len(full_q))
print(test_full_q[144])
print(full_q[144])

2000
11784
['show', 'the', 'whispering', 'willows', 'novel']
['please', 'add', 'liberty', 'forever', 'to', 'the', 'road', 'trip', 'playlist', '.']


In [60]:
def get_vocab(full_q):
    vocab = {}
    for i in full_q:
        for j in i:
            if(j not in vocab):
                vocab[j] = 1
            else:
                vocab[j] += 1
    return vocab
vocab = get_vocab(full_q)
print(vocab)

{'add': 1372, 'another': 49, 'song': 374, 'to': 2939, 'the': 6366, 'cita': 1, 'romantica': 4, 'playlist': 1151, '.': 2021, 'clem': 2, 'burke': 5, 'in': 2964, 'my': 1206, 'pre-party': 8, 'r': 26, '&': 118, 'b': 49, 'jams': 13, 'live': 57, 'from': 907, 'aragon': 2, 'ballroom': 1, 'trapeo': 4, 'unite': 2, 'and': 807, 'win': 2, 'night': 61, 'out': 631, 'piano': 20, 'bar': 99, 'cindy': 1, 'wilson': 6, 'spanish': 11, 'harlem': 3, 'incident': 2, 'cleaning': 7, 'house': 249, 'greyest': 1, 'of': 2204, 'blue': 15, 'skies': 4, 'indie': 52, 'espanol': 12, 'name': 33, 'kids': 15, 'street': 19, 'plylist': 1, 'new': 159, 'mix': 7, 'album': 404, 'radar': 3, 'latino': 11, 'tranquility': 1, 'latin': 37, 'pop': 49, 'rising': 12, 'd': 4, 'flame': 5, 'dcode': 2, 'fairy': 8, 'tales': 11, 'i': 1900, 'love': 109, 'neo': 5, 'soul': 43, 'national': 101, 'treasure': 2, 'book': 1375, 'secrets': 9, 'fire': 29, 'wind': 22, 'digster': 9, 'future': 12, 'hits': 53, 'caleigh': 2, 'peters': 4, 'women': 52, 'country': 49

In [61]:
closed_vocab = []
for i in vocab:
    if(vocab[i] > 1):
        closed_vocab.append(i)

In [ ]:
new_full_q = []
for i in full_q:
    new = []
    for j in i:
        if(j in closed_vocab):
            new.append(j)
        else:
            new.append("<unk>")
    new_full_q.append(new)
full_q = new_full_q
full_q

In [ ]:
def n_gram_fun(sentences , n):
    n_grams = {}
    for se in sentences:
        se = ["<s>"] * (n-1) + se + ["</s>"]
        se = tuple(se)
        for i in range(len(se)):
            gram = se[i:i+n]

            if gram in n_grams:         
                n_grams[gram] += 1
            else:      
                n_grams[gram] = 1
    return n_grams


print(n_gram_fun(full_q ,2))
#print(n_gram_fun([['add', 'another', 'song', 'to', 'the', 'cita', 'romantica', 'playlist', '.']] , 1))
#p(y|x) = c(x y )/c(x)
#p(another | add) = c(add another)

In [64]:
# p(word|prev_word) = 
def probability(word, prev_words, 
                         count_n_gram, count_one_n_gram, v_siz , k=1):
  
    prev_words = prev_words.lower()
    prev_words = nltk.word_tokenize(prev_words)
    prev_word = tuple(prev_words)
   
    
    
    prev_word_count = 0
    if(prev_word in count_n_gram):
        prev_word_count = count_n_gram[prev_word]
  
    denominator = prev_word_count + k * v_siz
    prev_words.append(word)
    n_1_gram =  tuple(prev_words)
    
    whole_word_count = 0
    if(n_1_gram in count_one_n_gram):
        whole_word_count = count_one_n_gram[n_1_gram]
    
    numerator = whole_word_count + k
  
    probability = numerator / denominator
    
    
    return probability

In [ ]:
trigram = n_gram_fun(full_q, 3)
bigram = n_gram_fun(full_q, 2)
print(bigram)
prob = probability("clem", "add", bigram,trigram, len(closed_vocab), k =1.0)
print(prob)

In [67]:
def all_probability(pre_words, prefix,count_n_gram, count_one_n_gram, v , k=1):
  
    
    v_siz = len(v)
    
    pro = {}
    
    for word in v:
        if(len(prefix) > len(word) and prefix != '-1'):
            continue
        index = len(prefix)
        if(word[0:index] != prefix and prefix != '-1'):
            continue

        probabilit = probability(word,pre_words, 
                                   count_n_gram, count_one_n_gram, 
                                   v_siz, k)
        pro[word] = probabilit

    return pro

In [68]:
words = input("enter sentence\n")
pre = input("enter prefix or -1 to ignore\n")
pre = pre.lower()

enter sentence
add
enter prefix or -1 to ignore
-1


In [69]:
prob = all_probability(words, pre ,unigram, bigram, closed_vocab, 1.0)
#print(prob)
new_prob = {}

ls_ans = []
for i in prob :
    ls_ans.append(prob[i])
    new_prob[prob[i]] = i
ls_ans.sort(reverse =True)
for i in range(min(len(ls_ans) , 10)):
    print(new_prob[ls_ans[i]])

this
the
a
tune
artist
track
song
an
album
album


In [70]:
def preprexity(test_q, n):
    ans = 1
    new_test_q =[]
    for i in test_q:
        ls = []
        for j in i:
            if(j not in closed_vocab):
                ls.append('<unk>')
            else:
                ls.append(j)
        new_test_q.append(ls)
    test_q = new_test_q
    v_siz = len(vocab)
    n_gram = n_gram_fun(test_q , n)
    
    n_one_gram = n_gram_fun(test_q,n+1)
    
    for i in test_q:
        i = ["<s>"]*n+i+["</s>"]
        i = i[::-1]
        # <s> h e l l o </s>
        #</s> o l l e h <s>
        p = 0
        while(p+n+1 <= len(i)):
            word = str(i[p:p+1])
            prev_word = str(i[p+1:p+n+1][::-1])
            ans +=  math.log(probability(word, prev_word, 
                         n_gram, n_one_gram, v_siz , k=1) , 2)
            p +=1
    
       
    ans *= -1/len(test_q)
    return ans

In [71]:
j = preprexity(test_full_q , 1)
print(j)

132.8947380491337
